<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/onestop_data_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

Mounted at /gdrive


In [0]:
%cd "/gdrive/My Drive/tensor-flow-state/tensor-flow-state"

/gdrive/My Drive/tensor-flow-state/tensor-flow-state


In [0]:
import pandas as pd
import numpy as np

In [0]:
data_dir = "data/"
# Define sensors to process
sensor_name_list = ["RWS01_MONIBAS_0021hrl0414ra", "RWS01_MONIBAS_0021hrl0403ra", "RWS01_MONIBAS_0021hrl0409ra", "RWS01_MONIBAS_0021hrl0420ra", "RWS01_MONIBAS_0021hrl0426ra"]

### ------------------------------------------------------------ START OF MESSING AROUND ------------------------------------------------------------

### ------------------------------------------------------------ END OF MESSING AROUND ------------------------------------------------------------

### Clean sensor data

In [0]:
import datetime
def dateparse (time_in_secs):
    # Unix/epoch time to "YYYY-MM-DD HH:MM:SS"
    return datetime.datetime.fromtimestamp(float(time_in_secs))

def repair_datetime_index(df, freq = "T"):
    df = df.loc[~df.index.duplicated(keep = "first")] # remove duplicate date time indexes
    df = df.reindex(pd.date_range(start = df.index.min(), end = df.index.max(), freq = freq)) # add missing date time indexes
    df.index = df.index.tz_localize("UTC").tz_convert("Europe/Amsterdam")

    return df

def fix_values(df):
    # The order of these operations is currently important! Pay attention when making changes
    df["speed_limit"] = np.where((df.index.hour < 19) & (df.index.hour >= 6), 100, 130)
    df.loc[df.flow < 0, "flow"] = np.nan # flow is either -2 (missing data) or 0 or positive. -2 to nan
    df.loc[df.speed < -1, "speed"] = np.nan # -2 (missing data) as well as oddities (-1.33, an average over -2 and -1 lanes?) to nan 
    df.speed.mask(df.speed == -1, df.speed_limit, inplace = True) # -1 means no cars, setting it to speed limit
    df.loc[(df.speed < 0) & (df.speed > -1), "speed"] = 0 # anything else below zero is between 0 and -1, occuring when some lanes have non-moving cars while others have have no cars.
    df.speed.mask(df.speed > df.speed_limit, df.speed_limit, inplace = True) # cap speed at speed_limit, since higher speed dosn't add to representation
    
    return df

import os
def reduce_cols(sensors, path_in = "data/ndw_raw/", path_out = "data/"):
    sensor_df_list = list()
    for sensor in sensors:
        df = pd.read_csv(os.path.join(path_in, sensor + ".csv"), header = None, \
                         usecols = [0, 86, 87], names = ["timestamp", "speed", "flow"], \
                         index_col = "timestamp", parse_dates = True, date_parser = dateparse)
        df.flow /= 60 # change flow unit to min^-1
        df = repair_datetime_index(df)
        df = fix_values(df)        
        #df.to_csv(path_out + sensor)
        sensor_df_list.append(df)

    return sensor_df_list

In [0]:
sensor_df_list = reduce_cols(sensor_name_list)

### Join Sensors

In [0]:
def join_sensors(sensor_df_list, sensor_name_list):
    combined_df = pd.DataFrame({"timestamp": pd.date_range(start = "2011-01-01", end = "2019-12-31", freq = "T")})
    combined_df.set_index("timestamp", drop = True, inplace = True)
    combined_df.index = combined_df.index.tz_localize("UTC").tz_convert("Europe/Amsterdam")
    d = {}
    for i, sensor in enumerate(sensor_df_list):
        # only add speed limit on the final sensor
        if i == len(sensor_df_list) - 1:
            d[sensor_name_list[i]] = sensor_df_list[i]
            combined_df = combined_df.join(d[sensor_name_list[i]], how = "outer", rsuffix = '_' + sensor_name_list[i])
        else:
            d[sensor_name_list[i]] = sensor_df_list[i].iloc[:, :2]
            combined_df = combined_df.join(d[sensor_name_list[i]], how = "outer", rsuffix = "_" + sensor_name_list[i])
    combined_df.dropna(how = "all", axis = 0, inplace = True) # this works in all cases because speed_limit is never NA on a sensor df
    
    return combined_df

In [0]:
# Join sensors to one table
df = join_sensors(sensor_df_list, sensor_name_list)

# Rename and reorder columns
df.rename({"speed_RWS01_MONIBAS_0021hrl0403ra": "speed_-2", "speed_RWS01_MONIBAS_0021hrl0409ra": "speed_-1",\
           "speed_RWS01_MONIBAS_0021hrl0420ra": "speed_+1", "speed_RWS01_MONIBAS_0021hrl0426ra": "speed_+2",\
           "flow_RWS01_MONIBAS_0021hrl0403ra": "flow_-2", "flow_RWS01_MONIBAS_0021hrl0409ra": "flow_-1",\
           "flow_RWS01_MONIBAS_0021hrl0420ra": "flow_+1", "flow_RWS01_MONIBAS_0021hrl0426ra": "flow_+2"\
           }, axis = 1, inplace = True)
col_order = ["speed", "flow", "speed_-2", "speed_-1","speed_+1", "speed_+2", "flow_-2", "flow_-1", "flow_+1", "flow_+2", "speed_limit"]
df = df[col_order]

# Save table to csv
#df.to_csv(data_dir + "combined_df.csv")

In [0]:
df.head()

,speed,flow,speed_-2,speed_-1,speed_+1,speed_+2,flow_-2,flow_-1,flow_+1,flow_+2,speed_limit
2011-01-03 01:00:00+01:00,122.000000,8.0,100.000000,130.000000,118.5,129.750000,8.0,6.0,4.0,6.0,130.0
2011-01-03 01:01:00+01:00,130.000000,5.0,115.000000,126.333333,118.0,104.000000,18.0,6.0,5.0,2.0,130.0
2011-01-03 01:02:00+01:00,111.000000,2.0,112.666667,104.000000,91.0,112.666667,14.0,2.0,4.0,8.0,130.0
2011-01-03 01:03:00+01:00,105.333333,10.0,118.000000,108.666667,120.0,130.000000,14.0,6.0,9.0,10.0,130.0
2011-01-03 01:04:00+01:00,130.000000,8.0,105.500000,121.000000,127.5,117.500000,21.0,13.0,7.0,5.0,130.0


### Impute data

In [0]:
cols = col_order
speed_cols = ["speed", "speed_-2", "speed_-1","speed_+1", "speed_+2"]
flow_cols = ["flow", "flow_-2", "flow_-1", "flow_+1", "flow_+2"]

In [0]:
# Where values are missing in one or more sensors, but are present in others, impute with mean of others
def fill_na_row_mean(df):
    row_avgs = df.mean(axis = 1).values.reshape(-1, 1)
    df = df.fillna(0) + df.isna().values * row_avgs
    return df

In [0]:
speed_df = fill_na_row_mean(df[speed_cols])
flow_df = fill_na_row_mean(df[flow_cols])

In [0]:
df = speed_df.join(flow_df, how = "inner").join(df[["speed_limit"]], how = "inner")

In [0]:
# Interpolate null vals for the first week of data of speed and flow cols
def interpolate_week(df, cols):
    week = 7 * 24 * 60
    for col in cols: 
        df.iloc[:week, df.columns.get_loc(col)] = df[col][:week].interpolate(method = "time")
    return df

# Replace remaining nulls with value from 1 week previous
def shift_week(df, cols):
    # Use RangeIndex for the this operation
    df["timestamp"] = df.index
    df.reset_index(drop = True, inplace = True)
    week = 7 * 24 * 60
    for col in cols:
        col_index = df.columns.get_loc(col)
        for row in df.itertuples():
            if np.isnan(row[col_index + 1]):
                df.iat[row[0], col_index] = df.iat[(row[0] - week), col_index]
    # Return to DateTimeIndex again
    df.set_index(pd.to_datetime(df.timestamp.values), inplace = True) 
    df.drop("timestamp", axis = 1, inplace = True)
    return df

In [0]:
df = interpolate_week(df, cols)
df = shift_week(df, cols)
#df.to_csv("data/df_imputed_week_shift.csv")

In [0]:
import holidays
df["density"] = (df.flow * 60) / df.speed
df["weekend"] = np.where(df.index.weekday > 4, 1, 0).astype(np.int16)
df["holiday"] = np.array([int(x in holidays.NL()) for x in df.index]).astype(np.int16)
df["speed_limit"] =  np.where(df.speed_limit > 115, 1, 0)

In [0]:
df.to_csv("data/df_imputed_week_shift_added_holiday_weekends_speed_limit_130.csv")